# Introduction

The goal of this notebook is to show how [SharpPlot](https://www.sharpplot.com/) can be used for the production of high quality graphs in Dyalog jupyter notebooks. It is shown, as an example, how to create a nice looking graph with a custom format. However, the notebook is not intended to be a comprehensive tutorial on the usage of SP. For further details, refer to the [official documentation](https://www.sharpplot.com/SharpPlot.htm) (take into account that this documentation is written for the C# version of SP; check the [Rosetta stone](https://www.sharpplot.com/Languages.htm) to see how to translate these commands into Dyalog APL).

In [1]:
)clear   ⍝ begin on a clean workspace
⎕IO←0    ⍝ for convenience

clear ws

## Sample data

Let's generate some sample data to have something to work with. Initially, we use the user command [`]plot`](https://github.com/Dyalog/dyalog-jupyter-kernel/wiki/1.1-Creating-content#rich-content) to display the data.

In [2]:
y1 y2 ← 1 2 ○⊂ x ← 0.1×⍳100 ⍝ sin and cos
]plot (y1 y2) x

Created by Causeway SVG engine - SharpPlot v3.71.0 
 
 Paint the paper ===== 
 
 
 
 Border ===== 
 Region ===== 
 X-Axis Ticks ===== 
 X-Axis tickmarks 
 
 Y-Axis Ticks ===== 
 Y-Axis tickmarks 
 
 Axes ===== 
 
 Y-axis labels 
 
 -1 
 -0.8 
 -0.6 
 -0.4 
 -0.2 
 0 
 0.2 
 0.4 
 0.6 
 0.8 
 1 
 
 for X-axis labels 
 
 0 
 1 
 2 
 3 
 4 
 5 
 6 
 7 
 8 
 9 
 10 
 
 Heading, subheading and footnotes ===== 
 Start of Line Chart =========== 
 Points follow ... 
 Line 
 
 <path d="M4063 2107L4026 2004 3989 1897 3952 1787 3915 1675 3878 1562 3841 1449 3804 1338 3767 1229 3730 1124 3693 1024 3656 929 3619 841 3582 760 3545 688 3508 624 3471 571 3434 527 3397 494 3360 472 3323 461 3286 462 3249 473 3212 496 3175 530 3138 574 3101 629 3064 693 3027 766 2990 848 2953 936 2916 1032 2879 1133 2842 1238 2805 1347 2768 1458 2731 1571 2694 1684 2657 1796 2620 1906 2583 2012 2546 2115 2509 2212 2472 2303 2435 2387 2398 2463 2361 2530 2324 2588 2287 2636 2250 2674 2213 2700 2176 2716 2139 2720 2102 2713 2065 2695 2028 2665 1991 2625 1954 2575 1917 2515 1880 2445 1843 2367 1806 2281 1769 2189 1732 2090 1695 1986 1658 1879 1621 1768 1584 1656 1547 1543 1510 1431 1473 1320 1436 1211 1399 1107 1362 1007 1325 914 1288 827 1251 747 1214 676 1177 615 1140 562 1103 521 1066 490 1029 469 992 460 955 463 918 476 881 501 844 537 807 583 770 639 733 705 696 779 659 862 622 952 585 1048 548 1150 511 1256 474 1366 437 1477 400 1590" />
 
 Points follow ... 
 Line 
 
 <path d="M4063 2595L4026 2641 3989 2677 3952 2703 3915 2717 3878 2720 3841 2711 3804 2692 3767 2661 3730 2620 3693 2568 3656 2507 3619 2436 3582 2357 3545 2270 3508 2177 3471 2077 3434 1973 3397 1865 3360 1754 3323 1642 3286 1529 3249 1417 3212 1306 3175 1198 3138 1094 3101 996 3064 903 3027 816 2990 738 2953 668 2916 608 2879 557 2842 516 2805 486 2768 468 2731 460 2694 464 2657 479 2620 505 2583 542 2546 589 2509 647 2472 714 2435 789 2398 873 2361 964 2324 1061 2287 1163 2250 1269 2213 1379 2176 1491 2139 1604 2102 1717 2065 1828 2028 1937 1991 2043 1954 2144 1917 2240 1880 2329 1843 2410 1806 2484 1769 2548 1732 2603 1695 2648 1658 2682 1621 2706 1584 2718 1547 2719 1510 2709 1473 2687 1436 2655 1399 2612 1362 2558 1325 2495 1288 2423 1251 2343 1214 2255 1177 2160 1140 2060 1103 1955 1066 1847 1029 1736 992 1623 955 1510 918 1398 881 1288 844 1181 807 1077 770 979 733 888 696 803 659 726 622 657 585 598 548 549 511 510 474 483 437 466 400 460" />
 
 Reset to original origin

The `]plot` command only allows very basic functionality (displaying functions with a common x-axis using different plotting methods), with no configuration options. For instance, if we have different "sampling rates", we won't be able to use `]Plot` to create a single plot.

In [3]:
y1 ← 1 ○ x1 ← 0.1×⍳100           ⍝ sin
y2 ← 2 ○ x2 ← (0.02×○3)×⍳50      ⍝ cos
data ← sin cos ← (y1 x1) (y2 x2) ⍝ data to plot

In order to create more advanced graphs, we will use SP. First, we will do it directly in the workspace, and at the end we will define our own `Plot` function.

# Plotting

## Initialization

The `sharpplot` workspace provides a `InitCauseway` function that will initialize everything required to work with SP from Dyalog APL (as well as some additional functionality which won't be discussed here). After running it, we will have a `Causeway` namespace. Then, we create a new instance of the `Causeway.SharpPlot` class, that we will use to interact with SP.

In [4]:
⍝ Init
'InitCauseway'⎕CY'sharpplot'
InitCauseway ⍬
sp←⎕NEW Causeway.SharpPlot

## Formatting

Several `sp` fields and methods can be used to adjust the format of our plot. We will define the format of gridlines, labels, captions, and lines. A large number of formatting options are available. The [SP User Manual](https://www.sharpplot.com/Styles.htm) contains more information on format and styles.

In [5]:
⍝ Format
sp.XAxisStyle←Causeway.XAxisStyles.GridLines
sp.YAxisStyle←Causeway.YAxisStyles.(GridLines+AtEndCaption)
sp.SetKeyFont 9.5 System.Drawing.Color.Black
sp.SetXCaptionFont 9.5 System.Drawing.Color.Black
sp.SetYCaptionFont 9.5 System.Drawing.Color.Black
sp.SetXLabelFont 7.5 System.Drawing.Color.Black
sp.SetYLabelFont 7.5 System.Drawing.Color.Black
sp.XLabelFormat←'0.00'
sp.YLabelFormat←'0.00'
sp.SetPenWidths 2
sp.SetColors ⊂'black'

## Ranges and labels

Next, we set up the key and axis labels, as well as the limits of the plot. In order to define these limits, we need to find the maximum and minimum values in all the curves:

In [6]:
⍝ Labels
sp.XCaption←,'x'
sp.YCaption←,'y',⎕UCS 10 ⍝ add newline to leave some space
sp.SetKeyText⊂'sin' 'cos'

In [7]:
⍝ Limits
ya xa←⌊⌿⌊/¨↑data
yw xw←⌈⌿⌈/¨↑data
(xa xw) (ya yw)
sp.SetYRange ya yw
sp.SetXRange xa xw

┌─────┬───────────────┐
│0 9.9│¯0.9999232576 1│
└─────┴───────────────┘

## Plot and display

To plot, we use the [`sp.DrawLineGraph`](https://www.sharpplot.com/SharpPlot-DrawLineGraph.htm) function. 

In [8]:
⍝ Plot
sp.DrawLineGraph¨data

The plot is already created, but we still need to display it. Alternatively, we could [save it as an image](https://www.sharpplot.com/SharpPlot-SaveImage.htm) or [pdf](https://www.sharpplot.com/SharpPlot-SaveImage.htm) file, for example. To render the plot inside the jupyter notebook, we first generate a SVG graph with [`sp.RenderSvg`](https://www.sharpplot.com/SharpPlot-RenderSvg.htm). Then, the I-beam command [`3500⌶`](https://help.dyalog.com/latest/Content/Language/I%20Beam%20Functions/Send%20Text%20to%20RIDE-embedded%20Browser.htm#SendTexttoRIDEembeddedBrowser) sends the generated SVG to the browser.

In [9]:
⍝ Display
3500⌶sp.RenderSvg Causeway.SvgMode.FixedAspect

Created by Causeway SVG engine - SharpPlot v3.71.0 
 
 Paint the paper ===== 
 
 
 
 Border ===== 
 Region ===== 
 X-Axis Ticks ===== 
 X-Axis Grid 
 
 X-Axis tickmarks 
 
 Y-Axis Ticks ===== 
 Y grid 
 
 Y-Axis tickmarks 
 
 Axes ===== 
 
 Y-axis labels 
 
 -1.00 
 -0.80 
 -0.60 
 -0.40 
 -0.20 
 0.00 
 0.20 
 0.40 
 0.60 
 0.80 
 1.00 
 
 for X-axis labels 
 
 0.00 
 1.00 
 2.00 
 3.00 
 4.00 
 5.00 
 6.00 
 7.00 
 8.00 
 9.00 
 10.00 
 
 Y-Captions: text and effects 
 
 y 
 
 X-Caption: text and effects 
 
 x 
 
 Heading, subheading and footnotes ===== 
 Start of Line Chart =========== 
 Points follow ... 
 Line 
 
 <path d="M4063 2107L4026 2004 3989 1897 3952 1787 3915 1675 3878 1562 3841 1449 3804 1338 3767 1229 3730 1124 3693 1024 3656 929 3619 841 3582 760 3545 688 3508 624 3471 571 3434 527 3397 494 3360 472 3323 461 3286 462 3249 473 3212 496 3175 530 3138 574 3101 629 3064 693 3027 766 2990 848 2953 936 2916 1032 2879 1133 2842 1238 2805 1347 2768 1458 2731 1571 2694 1684 2657 1796 2620 1906 2583 2012 2546 2115 2509 2212 2472 2303 2435 2387 2398 2463 2361 2530 2324 2588 2287 2636 2250 2674 2213 2700 2176 2716 2139 2720 2102 2713 2065 2695 2028 2665 1991 2625 1954 2575 1917 2515 1880 2445 1843 2367 1806 2281 1769 2189 1732 2090 1695 1986 1658 1879 1621 1768 1584 1656 1547 1543 1510 1431 1473 1320 1436 1211 1399 1107 1362 1007 1325 914 1288 827 1251 747 1214 676 1177 615 1140 562 1103 521 1066 490 1029 469 992 460 955 463 918 476 881 501 844 537 807 583 770 639 733 705 696 779 659 862 622 952 585 1048 548 1150 511 1256 474 1366 437 1477 400 1590" />
 
 Start of Line Chart =========== 
 Points follow ... 
 Line 
 
 
 
 Key ===== 
 Key - Line 
 
 
 
 Key - Line 
 
 
 
 
 sin 
 
 
 cos 
 
 Reset to original origin

0

The expression above should display the plot of the two sample curves and return `0`. If some error happened when displaying the plot, the result will be `¯1`.

# Plotting ~~function~~ operator

To finish, we are going to put everything together. We want to define a plotting function, but we will need more than two parameters, so we define a custom operator instead. There are many options that could be added to such a function or operator. The intention is not to provide a very complete and configurable implementation, but just to show how to define your own.

The `Plot` operator (a [tradop](https://help.dyalog.com/latest/index.htm#Language/Defined%20Functions%20and%20Operators/TradFns/Introduction.htm)) takes two operands, which will be the labels of the x and y axis, a dataset as right argument and, optionally, a list of colors as left argument. The dataset consists of a list of curves, where each curve is composed of a name and yx data. We handle as a special case a single curve (`¯4<≡data`) and apply "scalar extension" to the left argument (`(≢data)⍴⊆colors`) so that the user can easily specify a common color for all curves. In order to make it possible to set an individual color for each curve, we plot them inside a `:For` loop (using a dfn or a tacit function is not possible because the SP SetColors method does not return anything).

In [10]:
]dinput
{R}←{colors}(xlabel Plot ylabel) data;sp;names;x;y;InitCauseway;Causeway;System
⍝ eg: 'red' 'blue' ('xlabel'Plot'ylabel') ('series1' (y1 x1)) ('series2' (y2 x2))

⍝ Init
'InitCauseway'⎕CY'sharpplot'
InitCauseway ⍬
sp←⎕NEW Causeway.SharpPlot

⍝ Format
sp.XAxisStyle←Causeway.XAxisStyles.GridLines
sp.YAxisStyle←Causeway.YAxisStyles.(GridLines+AtEndCaption)
sp.SetKeyFont 9.5 System.Drawing.Color.Black
sp.SetXCaptionFont 9.5 System.Drawing.Color.Black
sp.SetYCaptionFont 9.5 System.Drawing.Color.Black
sp.SetXLabelFont 7.5 System.Drawing.Color.Black
sp.SetYLabelFont 7.5 System.Drawing.Color.Black
sp.XLabelFormat←'0.00'
sp.YLabelFormat←'0.00'
sp.SetPenWidths 2

⍝ Curves
:If 0=⎕NC'colors'
    colors ← ⊂'black'
:EndIf
:If ¯4<≡data
    names data←data
    data←,⊂data
:Else
    names data←↓⍉↑data ⍝ names and data
:EndIf
colors ← (≢data)⍴⊆colors ⍝ "scalar extension"

⍝ Labels
sp.XCaption←,xlabel
sp.YCaption←,ylabel,⎕UCS 10 ⍝ add newline to leave some space
sp.SetKeyText⊂names

⍝ Limits
ya xa←⌊⌿⌊/¨↑data
yw xw←⌈⌿⌈/¨↑data
sp.SetYRange ya yw
sp.SetXRange xa xw

⍝ Plot and display
:For i :In ⍳≢data
    sp.SetColors i⌷colors 
    sp.DrawLineGraph i⊃data
:EndFor
R ← 3500⌶sp.RenderSvg Causeway.SvgMode.FixedAspect

In [11]:
⍝ eg
'red' 'blue' ('x'Plot'y') ('sin' sin) ('cos' cos)                                ⍝ sin and cos, with colors
'purple' ('x'Plot'y') 'log' ((⍟1↓x) x)                                           ⍝ single curve
('Time [s]'Plot'Distance [m]') ('a = 20' ((x×100+10×x) x)) ('a = 0' ((x×100) x)) ⍝ default color

Created by Causeway SVG engine - SharpPlot v3.71.0 
 
 Paint the paper ===== 
 
 
 
 Border ===== 
 Region ===== 
 X-Axis Ticks ===== 
 X-Axis Grid 
 
 X-Axis tickmarks 
 
 Y-Axis Ticks ===== 
 Y grid 
 
 Y-Axis tickmarks 
 
 Axes ===== 
 
 Y-axis labels 
 
 -1.00 
 -0.80 
 -0.60 
 -0.40 
 -0.20 
 0.00 
 0.20 
 0.40 
 0.60 
 0.80 
 1.00 
 
 for X-axis labels 
 
 0.00 
 1.00 
 2.00 
 3.00 
 4.00 
 5.00 
 6.00 
 7.00 
 8.00 
 9.00 
 10.00 
 
 Y-Captions: text and effects 
 
 y 
 
 X-Caption: text and effects 
 
 x 
 
 Heading, subheading and footnotes ===== 
 Start of Line Chart =========== 
 Points follow ... 
 Line 
 
 <path d="M4063 2107L4026 2004 3989 1897 3952 1787 3915 1675 3878 1562 3841 1449 3804 1338 3767 1229 3730 1124 3693 1024 3656 929 3619 841 3582 760 3545 688 3508 624 3471 571 3434 527 3397 494 3360 472 3323 461 3286 462 3249 473 3212 496 3175 530 3138 574 3101 629 3064 693 3027 766 2990 848 2953 936 2916 1032 2879 1133 2842 1238 2805 1347 2768 1458 2731 1571 2694 1684 2657 1796 2620 1906 2583 2012 2546 2115 2509 2212 2472 2303 2435 2387 2398 2463 2361 2530 2324 2588 2287 2636 2250 2674 2213 2700 2176 2716 2139 2720 2102 2713 2065 2695 2028 2665 1991 2625 1954 2575 1917 2515 1880 2445 1843 2367 1806 2281 1769 2189 1732 2090 1695 1986 1658 1879 1621 1768 1584 1656 1547 1543 1510 1431 1473 1320 1436 1211 1399 1107 1362 1007 1325 914 1288 827 1251 747 1214 676 1177 615 1140 562 1103 521 1066 490 1029 469 992 460 955 463 918 476 881 501 844 537 807 583 770 639 733 705 696 779 659 862 622 952 585 1048 548 1150 511 1256 474 1366 437 1477 400 1590" />
 
 Start of Line Chart =========== 
 Points follow ... 
 Line 
 
 
 
 Key ===== 
 Key - Line 
 
 
 
 Key - Line 
 
 
 
 
 sin 
 
 
 cos 
 
 Reset to original origin

Created by Causeway SVG engine - SharpPlot v3.71.0 
 
 Paint the paper ===== 
 
 
 
 Border ===== 
 Region ===== 
 X-Axis Ticks ===== 
 X-Axis Grid 
 
 X-Axis tickmarks 
 
 Y-Axis Ticks ===== 
 Y grid 
 
 Y-Axis tickmarks 
 
 Axes ===== 
 
 Y-axis labels 
 
 -2.50 
 -2.00 
 -1.50 
 -1.00 
 -0.50 
 0.00 
 0.50 
 1.00 
 1.50 
 2.00 
 2.50 
 
 for X-axis labels 
 
 0.00 
 1.00 
 2.00 
 3.00 
 4.00 
 5.00 
 6.00 
 7.00 
 8.00 
 9.00 
 10.00 
 
 Y-Captions: text and effects 
 
 y 
 
 X-Caption: text and effects 
 
 x 
 
 Heading, subheading and footnotes ===== 
 Start of Line Chart =========== 
 Points follow ... 
 Line 
 
 <path d="M4026 554L3989 558 3952 563 3915 568 3878 572 3841 577 3804 582 3767 587 3730 592 3693 597 3656 602 3619 607 3582 612 3545 617 3508 623 3471 628 3434 633 3397 639 3360 644 3323 650 3286 656 3249 662 3212 667 3175 673 3138 679 3101 685 3064 691 3027 698 2990 704 2953 710 2916 717 2879 724 2842 730 2805 737 2768 744 2731 751 2694 758 2657 765 2620 773 2583 780 2546 788 2509 795 2472 803 2435 811 2398 819 2361 828 2324 836 2287 845 2250 854 2213 863 2176 872 2139 881 2102 891 2065 900 2028 910 1991 920 1954 931 1917 941 1880 952 1843 963 1806 975 1769 987 1732 999 1695 1011 1658 1024 1621 1037 1584 1050 1547 1064 1510 1079 1473 1093 1436 1109 1399 1125 1362 1141 1325 1158 1288 1176 1251 1194 1214 1214 1177 1234 1140 1255 1103 1277 1066 1300 1029 1324 992 1350 955 1378 918 1407 881 1438 844 1471 807 1508 770 1547 733 1590 696 1638 659 1691 622 1751 585 1821 548 1903 511 2004 474 2134 437 2317 400 2631" />
 
 Key ===== 
 Key - Line 
 
 
 
 
 log 
 
 Reset to original origin

Created by Causeway SVG engine - SharpPlot v3.71.0 
 
 Paint the paper ===== 
 
 
 
 Border ===== 
 Region ===== 
 X-Axis Ticks ===== 
 X-Axis Grid 
 
 X-Axis tickmarks 
 
 Y-Axis Ticks ===== 
 Y grid 
 
 Y-Axis tickmarks 
 
 Axes ===== 
 
 Y-axis labels 
 
 0.00 
 200.00 
 400.00 
 600.00 
 800.00 
 1000.00 
 1200.00 
 1400.00 
 1600.00 
 1800.00 
 2000.00 
 
 for X-axis labels 
 
 0.00 
 1.00 
 2.00 
 3.00 
 4.00 
 5.00 
 6.00 
 7.00 
 8.00 
 9.00 
 10.00 
 
 Y-Captions: text and effects 
 
 Distance [m] 
 
 X-Caption: text and effects 
 
 Time [s] 
 
 Heading, subheading and footnotes ===== 
 Start of Line Chart =========== 
 Points follow ... 
 Line 
 
 <path d="M4063 494L4026 527 3989 561 3952 594 3915 627 3878 659 3841 692 3804 724 3767 756 3730 788 3693 819 3656 851 3619 882 3582 912 3545 943 3508 973 3471 1004 3434 1034 3397 1063 3360 1093 3323 1122 3286 1151 3249 1180 3212 1209 3175 1237 3138 1265 3101 1293 3064 1321 3027 1348 2990 1375 2953 1402 2916 1429 2879 1456 2842 1482 2805 1508 2768 1534 2731 1560 2694 1585 2657 1610 2620 1635 2583 1660 2546 1684 2509 1709 2472 1733 2435 1757 2398 1780 2361 1804 2324 1827 2287 1850 2250 1873 2213 1895 2176 1917 2139 1939 2102 1961 2065 1983 2028 2004 1991 2025 1954 2046 1917 2067 1880 2087 1843 2107 1806 2127 1769 2147 1732 2167 1695 2186 1658 2205 1621 2224 1584 2243 1547 2261 1510 2279 1473 2297 1436 2315 1399 2333 1362 2350 1325 2367 1288 2384 1251 2400 1214 2417 1177 2433 1140 2449 1103 2465 1066 2480 1029 2495 992 2510 955 2525 918 2540 881 2554 844 2568 807 2582 770 2596 733 2609 696 2622 659 2635 622 2648 585 2661 548 2673 511 2685 474 2697 437 2709 400 2720" />
 
 Start of Line Chart =========== 
 Points follow ... 
 Line 
 
 <path d="M4063 1601L4026 1613 3989 1624 3952 1635 3915 1647 3878 1658 3841 1669 3804 1680 3767 1692 3730 1703 3693 1714 3656 1726 3619 1737 3582 1748 3545 1760 3508 1771 3471 1782 3434 1793 3397 1805 3360 1816 3323 1827 3286 1839 3249 1850 3212 1861 3175 1873 3138 1884 3101 1895 3064 1906 3027 1918 2990 1929 2953 1940 2916 1952 2879 1963 2842 1974 2805 1986 2768 1997 2731 2008 2694 2019 2657 2031 2620 2042 2583 2053 2546 2065 2509 2076 2472 2087 2435 2099 2398 2110 2361 2121 2324 2132 2287 2144 2250 2155 2213 2166 2176 2178 2139 2189 2102 2200 2065 2212 2028 2223 1991 2234 1954 2245 1917 2257 1880 2268 1843 2279 1806 2291 1769 2302 1732 2313 1695 2325 1658 2336 1621 2347 1584 2358 1547 2370 1510 2381 1473 2392 1436 2404 1399 2415 1362 2426 1325 2438 1288 2449 1251 2460 1214 2471 1177 2483 1140 2494 1103 2505 1066 2517 1029 2528 992 2539 955 2551 918 2562 881 2573 844 2584 807 2596 770 2607 733 2618 696 2630 659 2641 622 2652 585 2664 548 2675 511 2686 474 2697 437 2709 400 2720" />
 
 Key ===== 
 Key - Line 
 
 
 
 Key - Line 
 
 
 
 
 a = 20 
 
 
 a = 0 
 
 Reset to original origin